# 文件输入/输出

到目前为止，我们已经讨论了如何处理数据以及如何构建、训练和测试深度学习模型。然而，在某个时刻，我们可能会对学到的模型感到满意，并希望将结果保存下来以供日后在各种场景中使用（甚至可能是在部署中进行预测）。此外，在运行长时间的训练过程时，最佳做法是定期保存中间结果（检查点），以确保如果我们不小心绊到了服务器的电源线，不会丢失几天的计算成果。因此，现在是时候学习如何加载和存储单个权重向量和整个模型了。本节将解决这两个问题。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## （加载和保存张量）

对于单个张量，我们可以直接
调用 `load` 和 `save` 函数
分别进行读取和写入。
这两个函数都要求我们提供一个名称，
并且 `save` 还需要输入要保存的变量。

In [2]:
x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。

In [3]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表并将它们读回内存。**]

In [4]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以[**编写和读取一个将字符串映射到张量的字典。**]
当我们想要读取或写入模型中的所有权重时，这非常方便。

In [5]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）是有用的，
但如果我们想保存
（并在以后加载）整个模型，这会变得非常繁琐。
毕竟，我们可能有数百个
分散在各处的参数组。
因此，深度学习框架提供了内置功能
来加载和保存整个网络。
需要注意的一个重要细节是，这
保存的是模型*参数*而不是整个模型。
例如，如果我们有一个3层的MLP，
我们需要单独指定架构。
这样做的原因是模型本身可以包含任意代码，
因此它们不能自然地被序列化。
因此，为了恢复一个模型，我们需要
通过代码生成架构
然后从磁盘加载参数。
(**让我们从熟悉的MLP开始。**)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们将模型的参数存储为名为"mlp.params"的文件。

In [7]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们实例化原始MLP模型的一个克隆。
我们没有随机初始化模型参数，
而是[**直接读取文件中存储的参数**]。

In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，对于相同的输入`X`，计算结果应该是相同的。让我们来验证这一点。

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 摘要

`save` 和 `load` 函数可用于执行张量对象的文件输入输出。
我们可以通过参数字典保存和加载整个网络的参数集。
保存架构必须通过代码完成，而不能通过参数完成。

## 练习

1. 即使不需要将训练好的模型部署到不同的设备上，存储模型参数有哪些实际好处？
1. 假设我们只想重用网络的一部分，并将其集成到具有不同架构的新网络中。你将如何使用，比如说，先前网络中的前两层在新网络中？
1. 你将如何保存网络架构和参数？你会对架构施加哪些限制？

[讨论](https://discuss.d2l.ai/t/61)